# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from matplotlib.ticker import MaxNLocator

# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [2]:
portfolio_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [3]:
# portfolio_raw['channels'].explode().unique()

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [4]:
profile_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [5]:
transcript_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [6]:
transcript_raw['event'].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

In [7]:
{[*x][0] for x in transcript_raw['value']}

{'amount', 'offer id', 'offer_id'}

### Data Cleaning and Formatting Plan

#### **Objective**
- Clean and standardize the dataset while handling null values appropriately.

---

#### **Profile Dataset**
1. Split the `profile` dataset into two subsets:
   - One containing rows where the `gender` is `none`.
   - Another containing rows with all other `gender` values.

2. Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.

---

#### **Portfolio Dataset**
- Update column names to make them more descriptive and easier to read.

---

#### **Transcript Dataset**
- Modify the keys within the `value` dictionaries by replacing spaces (`' '`) with underscores (`'_'`) to ensure uniformity and avoid potential issues in further processing.

---

In [8]:
profile = profile_raw.copy(deep=True)

# Convert the 'became_member_on' column to a datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# Create a new column with only the year and month of the membership
profile['bec_memb_year_month'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d').dt.strftime('%Y-%m')

In [9]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   gender               14825 non-null  object        
 1   age                  17000 non-null  int64         
 2   id                   17000 non-null  object        
 3   became_member_on     17000 non-null  datetime64[ns]
 4   income               14825 non-null  float64       
 5   bec_memb_year_month  17000 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 797.0+ KB


In [10]:
portfolio = portfolio_raw.copy()
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [11]:
# Standardisation of column names
transcript = transcript_raw.copy(deep=True)
transcript['event'] = transcript['event'].str.replace(' ','_')

# Fixing the dicts key names
def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)

# Normalizing the value column and create a new dataframe
value_df = pd.json_normalize(transcript['value']) 
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

transcript_b['ofr_id_short'] = transcript_b['offer_id'].map(port_id)
transcript_b['ofr_id_short'] = transcript_b['ofr_id_short'].fillna('tran')
transcript_b['amount'] = transcript_b['amount'].fillna(0)
transcript_b['reward'] = transcript_b['reward'].fillna(0)

transcript_b = transcript_b.drop(columns = ['offer_id'])

transactions = transcript_b[transcript_b['ofr_id_short'] == 'tran'].copy()
events = transcript_b[transcript_b['ofr_id_short'] != 'tran'].copy()

transcript_b



,person,event,time,amount,reward,ofr_id_short
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,0.00,0.0,ofr_D
1,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0.00,0.0,ofr_E
2,e2127556f4f64592b11af22de27a7932,offer_received,0,0.00,0.0,ofr_J
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer_received,0,0.00,0.0,ofr_G
4,68617ca6246f4fbc85e91a2a49552598,offer_received,0,0.00,0.0,ofr_B
...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,0.0,tran
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,0.0,tran
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,0.0,tran
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,0.0,tran


In [12]:
transcript_b['event'].value_counts(dropna=False)

event
transaction        138953
offer_received      76277
offer_viewed        57725
offer_completed     33579
Name: count, dtype: int64

In [13]:
transactions

,person,event,time,amount,reward,ofr_id_short
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83,0.0,tran
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56,0.0,tran
12659,54890f68699049c2a04d415abc25e717,transaction,0,13.23,0.0,tran
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51,0.0,tran
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97,0.0,tran
...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,0.0,tran
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,0.0,tran
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,0.0,tran
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,0.0,tran


In [14]:
events

,person,event,time,amount,reward,ofr_id_short
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,0.0,0.0,ofr_D
1,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0.0,0.0,ofr_E
2,e2127556f4f64592b11af22de27a7932,offer_received,0,0.0,0.0,ofr_J
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer_received,0,0.0,0.0,ofr_G
4,68617ca6246f4fbc85e91a2a49552598,offer_received,0,0.0,0.0,ofr_B
...,...,...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer_completed,714,0.0,3.0,ofr_F
306506,b895c57e8cd047a8872ce02aa54759d6,offer_completed,714,0.0,2.0,ofr_G
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer_viewed,714,0.0,0.0,ofr_E
306509,8431c16f8e1d440880db371a68f82dd0,offer_completed,714,0.0,2.0,ofr_G


In [15]:
events

,person,event,time,amount,reward,ofr_id_short
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,0.0,0.0,ofr_D
1,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0.0,0.0,ofr_E
2,e2127556f4f64592b11af22de27a7932,offer_received,0,0.0,0.0,ofr_J
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer_received,0,0.0,0.0,ofr_G
4,68617ca6246f4fbc85e91a2a49552598,offer_received,0,0.0,0.0,ofr_B
...,...,...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer_completed,714,0.0,3.0,ofr_F
306506,b895c57e8cd047a8872ce02aa54759d6,offer_completed,714,0.0,2.0,ofr_G
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer_viewed,714,0.0,0.0,ofr_E
306509,8431c16f8e1d440880db371a68f82dd0,offer_completed,714,0.0,2.0,ofr_G


In [26]:
events_summary = events.groupby(['person', 'ofr_id_short']).agg(
    ev_cnt_events = ('event', 'count'),
    avg_ev_time = ('time', 'mean'),
    max_ev_time = ('time', 'max'),
    min_ev_time = ('time', 'min'),
    avg_amount_ev = ('amount', 'mean'),
    ev_reward_total = ('reward', 'sum'),
).reset_index()

transactions_summary = transactions.groupby(['person']).agg(
    tr_cnt_events = ('event', 'count'),
    avg_tr_time = ('time', 'mean'),
    max_tr_time = ('time', 'max'),
    min_tr_time = ('time', 'min'),
    avg_amount_tr = ('amount', 'mean'),
    tr_reward_total = ('reward', 'sum'),
).reset_index()

person_summary = events_summary.merge(transactions_summary, on='person', how='left')

person_summary_profile = person_summary.merge(profile, how='left', left_on='person', right_on='id') 

person_summary_profile.drop(columns=['id', 'became_member_on', 'bec_memb_year_month']).to_csv('person_summary_profile.csv')

In [22]:
events_summary

,person,ofr_id_short,ev_cnt_events,avg_ev_time,max_ev_time,min_ev_time,avg_amount_ev,ev_reward_total
0,0009655768c64bdeb2e877511632db8f,ofr_C,2,354.000000,372,336,0.0,0.0
1,0009655768c64bdeb2e877511632db8f,ofr_G,3,524.000000,540,504,0.0,2.0
2,0009655768c64bdeb2e877511632db8f,ofr_H,2,180.000000,192,168,0.0,0.0
3,0009655768c64bdeb2e877511632db8f,ofr_I,3,426.000000,456,408,0.0,5.0
4,0009655768c64bdeb2e877511632db8f,ofr_J,2,576.000000,576,576,0.0,2.0
...,...,...,...,...,...,...,...,...
63283,fffad4f4828548d1b5583907f2e9906b,ofr_I,6,246.000000,516,0,0.0,10.0
63284,ffff82501cea40309d5fdd7edcca4a07,ofr_D,3,514.000000,534,504,0.0,5.0
63285,ffff82501cea40309d5fdd7edcca4a07,ofr_E,3,180.000000,198,168,0.0,5.0
63286,ffff82501cea40309d5fdd7edcca4a07,ofr_G,3,22.000000,60,0,0.0,2.0


In [23]:
transactions_summary

,person,tr_cnt_trents,avg_tr_time,max_tr_time,min_tr_time,avg_amount_tr,tr_reward_total
0,0009655768c64bdeb2e877511632db8f,8,543.00,696,228,15.950000,0.0
1,00116118485d4dfda04fdbaba9a87b5c,3,408.00,474,294,1.363333,0.0
2,0011e0d4e6b944f998e987f904e8c1e5,5,451.20,654,132,15.892000,0.0
3,0020c2b971eb4e9188eac86d93036a77,8,348.75,708,54,24.607500,0.0
4,0020ccbbb6d84e358d3414a3ff76cffd,12,375.00,672,42,12.837500,0.0
...,...,...,...,...,...,...,...
16573,fff3ba4757bd42088c044ca26d73817a,11,246.00,552,6,52.816364,0.0
16574,fff7576017104bcc8677a8d63322b5e1,6,392.00,696,36,4.990000,0.0
16575,fff8957ea8b240a6b5e634b6ee8eafcf,5,379.20,576,18,2.430000,0.0
16576,fffad4f4828548d1b5583907f2e9906b,12,323.50,678,36,7.402500,0.0


In [24]:
person_summary

,person,ofr_id_short,ev_cnt_events,avg_ev_time,max_ev_time,min_ev_time,avg_amount_ev,ev_reward_total,tr_cnt_trents,avg_tr_time,max_tr_time,min_tr_time,avg_amount_tr,tr_reward_total
0,0009655768c64bdeb2e877511632db8f,ofr_C,2,354.000000,372,336,0.0,0.0,8.0,543.0,696.0,228.0,15.950000,0.0
1,0009655768c64bdeb2e877511632db8f,ofr_G,3,524.000000,540,504,0.0,2.0,8.0,543.0,696.0,228.0,15.950000,0.0
2,0009655768c64bdeb2e877511632db8f,ofr_H,2,180.000000,192,168,0.0,0.0,8.0,543.0,696.0,228.0,15.950000,0.0
3,0009655768c64bdeb2e877511632db8f,ofr_I,3,426.000000,456,408,0.0,5.0,8.0,543.0,696.0,228.0,15.950000,0.0
4,0009655768c64bdeb2e877511632db8f,ofr_J,2,576.000000,576,576,0.0,2.0,8.0,543.0,696.0,228.0,15.950000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63283,fffad4f4828548d1b5583907f2e9906b,ofr_I,6,246.000000,516,0,0.0,10.0,12.0,323.5,678.0,36.0,7.402500,0.0
63284,ffff82501cea40309d5fdd7edcca4a07,ofr_D,3,514.000000,534,504,0.0,5.0,15.0,340.0,648.0,60.0,15.071333,0.0
63285,ffff82501cea40309d5fdd7edcca4a07,ofr_E,3,180.000000,198,168,0.0,5.0,15.0,340.0,648.0,60.0,15.071333,0.0
63286,ffff82501cea40309d5fdd7edcca4a07,ofr_G,3,22.000000,60,0,0.0,2.0,15.0,340.0,648.0,60.0,15.071333,0.0
